https://iamtrask.github.io/2015/11/15/anyone-can-code-lstm/

In [0]:
import numpy as np

In [0]:
data = "hello"
data = data.lower()

vocabulary = list(set([c for c in data]))
vocabulary.sort()

def char2vec(c):
  return [1 if l==c else 0 for l in vocabulary]

def vec2char(v):
  return vocabulary[v.index(1)]

print(char2vec('e'))

[1, 0, 0, 0]


In [0]:
def sigmoid(x):
  return 1/(1 + np.exp(-x))
def sigmoid_derivative(x):
  return x*(1-x)

lr = 0.1
epochs = 10000
len_train = 4
i_dim = len(vocabulary)
h_dim = 32
o_dim = len(vocabulary)

syn_i = np.random.uniform(low=-1, high=1, size=[i_dim, h_dim])
syn_h = np.random.uniform(low=-1, high=1, size=[h_dim, h_dim])
syn_o = np.random.uniform(low=-1, high=1, size=[h_dim, o_dim])

for epoch in range(epochs):
  total_error = 0
  
  for i in range(len(data) - len_train):
    syn_i_update = np.zeros_like(syn_i)
    syn_h_update = np.zeros_like(syn_h)
    syn_o_update = np.zeros_like(syn_o)
    
    x_text = data[i:i+len_train]
    y_text = data[i+1:i+len_train+1]
    h_vals = []
    o_vals = []
    o_errors = []
    h_val_pre = [0]*h_dim
    
    for j in range(len_train):
      x = char2vec(x_text[j])
      y = char2vec(y_text[j])
      
      h_val = sigmoid(np.dot(x, syn_i) + np.dot(h_val_pre, syn_h))
      o_val = sigmoid(np.dot(h_val, syn_o))
      h_val_pre = h_val
      
      h_vals.append(h_val)
      o_vals.append(o_val)
      o_errors.append(y - o_val)
      
    h_delta_fut = [0]*h_dim 
    for j in range(len_train):
      x = char2vec(x_text[len_train-j-1])
      o_val = o_vals[len_train-j-1]
      h_val = h_vals[len_train-j-1]
      h_val_pre = h_vals[len_train-j-2]
      
      o_delta = o_errors[len_train-j-1] * sigmoid_derivative(o_val)
      h_delta = (np.dot(o_delta, syn_o.T) + np.dot(h_delta_fut, syn_h.T)) * sigmoid_derivative(h_val)
      h_delta_fut = h_delta
      
      r = np.dot(np.atleast_2d(h_val).T, np.atleast_2d(o_delta))
      syn_i_update += np.dot(np.atleast_2d(x).T, np.atleast_2d(h_delta))
      syn_h_update += np.dot(np.atleast_2d(h_val_pre).T, np.atleast_2d(h_delta))
      syn_o_update += np.dot(np.atleast_2d(h_val).T, np.atleast_2d(o_delta))
    
    syn_i += syn_i_update * len_train
    syn_h += syn_h_update * len_train
    syn_o += syn_o_update * len_train
   
  if(epoch % 1000 == 0): print(np.sum([np.abs(e) for e in o_errors]))

6.0128215314421904
0.05966629407044323
0.04173737738372951
0.0339090568939496
0.02926670502931334
0.02610822631370035
0.023781804357749837
0.0219768342052764
0.020524006632846183
0.019322136351558848


In [0]:
def predict(char, size):
  x = char2vec(char)
  text = char
  h_val_pre = [0]*h_dim
  
  for _ in range(size):
    h_val = sigmoid(np.dot(x, syn_i) + np.dot(h_val_pre, syn_h))
    o_val = sigmoid(np.dot(h_val, syn_o))
    
    idx = np.argmax(o_val)
    text += vocabulary[idx]
    
    h_val_pre = h_val
    x = o_val
  return text  

predict('h', 20)

'helloelloellolllolloo'